Imports

In [2]:
import numpy as np

Neural net architecture

In [5]:
n = [2, 3, 3, 1]
print("layer 0 / input layer size", n[0])
print("layer 1 size", n[1])
print("layer 2 size", n[2])
print("layer 3 size", n[3])

layer 0 / input layer size 2
layer 1 size 3
layer 2 size 3
layer 3 size 1


Randomly initialize the biases and weights.

In [ ]:
# Generates matrices rowsxcols with random floats
W1 = np.random.randn(n[1], n[0])
W2 = np.random.randn(n[2], n[1])
W3 = np.random.randn(n[3], n[2])
b1 = np.random.randn(n[1], 1)
b2 = np.random.randn(n[2], 1)
b3 = np.random.randn(n[3], 1)

In [10]:
print(W1)

[[ 0.40884794 -1.04696281]
 [-0.43146332 -0.46658375]
 [ 0.11740522  0.44781947]]


Training data

In [12]:
# Height and weight data for training
X = np.array([
    [150, 70],
    [254, 73],
    [312, 68],
    [120, 60],
    [154, 61],
    [212, 65],
    [216, 67],
    [145, 67],
    [184, 64],
    [130, 69]
])
print(X.shape)

# Transpose input data to match the shape for matrix multiplication
A0 = X.T
print(A0.shape) # prints (2, 10)

(10, 2)
(2, 10)
